In [11]:
from openai import OpenAI
import json

class Q_A_Generator:
  def __init__(self) -> None:
    self.client = OpenAI()
    
  def make_q_a(self, script, questioner, respondent, qeustion, num):
    completion = self.client.chat.completions.create(
      model="gpt-4-0125-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": f"너는 지금부터 온라인 강의의 스크립트를 보고 다음 조건에 맞게 질문과 답변을 주고받는 담화를 생성할 거야."+
        f"질문 유형: {qeustion}"+
        f"질문자: {questioner}"+
        f"답변자: {respondent}"+
        """결과물 형식: {"담화1": xxx, "담화2": xxx, ... , "담화n": xxx} json"""+
        """담화 형식: {"질문":xxx, "답변":xxx, "대답":xxx} 형식의 json"""+
        f"담화는 총 {num}개를 생성한다."+
        "질문과 답변은 질문자와 답변자의 정보에 맞는 말투여야 한다."+
        "학생과 선생님은 상대방의 나이와 수준에 알맞는 말투를 사용해야 한다."+
        "질문자와 답변자 모두 딱딱하지 않은 구어체를 사용한다."+
        "학습자는 교수자에게 반드시 존댓말을 써야 한다."+
        "질문은 서로 유사하지 않고 다양하고 창의적이어야 한다."
        },
        {"role": "user", "content": script}
      ]
    )
    return json.loads(completion.choices[0].message.content)
  
  def answer_for_question(self, script, questioner, respondent, qeustion):
    completion = self.client.chat.completions.create(
      model="gpt-4-0125-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": f"너는 지금부터 온라인 강의의 스크립트를 보고 질문자의 질문에 대해 다음 조건에 맞게 답변을 생성할 거야."+
        f"질문자: {questioner}"+
        f"답변자: {respondent}"+
        f"스크립트: {script}"+
        """결과물 형식: {"답변": xxx} json"""+
        "질문과 답변은 질문자와 답변자의 정보에 맞는 말투여야 한다."+
        "학생과 선생님은 상대방의 나이와 수준에 알맞는 말투를 사용해야 한다."+
        "질문자와 답변자 모두 딱딱하지 않은 구어체를 사용하고 존댓말을 써야 한다."+
        "질문은 서로 유사하지 않고 다양하고 창의적이어야 한다."
        },
        {"role": "user", "content": qeustion}
      ]
    )
    return json.loads(completion.choices[0].message.content)
  
  def answer_for_response(self, script, questioner, respondent, qeustion, response):
    completion = self.client.chat.completions.create(
      model="gpt-4-0125-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": f"너는 지금부터 온라인 강의의 스크립트를 보고 질문자의 질문에 대해 답변자가 답변을 했을 때 다시 질문자의 입장에서 답변이 정답인지 파악하고 그에 맞는 최종 대답을 생성해야 해."+
        f"질문자: {questioner}"+
        f"답변자: {respondent}"+
        f"스크립트: {script}"+
        """결과물 형식: {"정답여부":, "답변": xxx} json"""+
        "최종대답은 질문자가 답변자에게 하는 것이다."
        "질문과 답변은 질문자와 답변자의 정보에 맞는 말투여야 한다."+
        "학생과 선생님은 상대방의 나이와 수준에 알맞는 말투를 사용해야 한다."+
        "딱딱하지 않은 구어체를 사용하고 존댓말을 써야 한다."+
        "답변이 올바르다면 칭찬을 하고 상황에 따라 추가 설명을 덧붙인다."+
        "답변이 올바르지 않다면 올바른 설명을 해준다."
        },
        {"role": "user", "content": qeustion}
      ]
    )
    return json.loads(completion.choices[0].message.content)
  
  def summary(self, script):
    completion = self.client.chat.completions.create(
      model="gpt-4-0125-preview",
      messages=[
        {"role": "system", "content": "온라인 강의 스크립트를 주면 다음조건에 맞게 요약해줘"+
        "너에게 요약본을 넘겼을 때 전체 맥락을 고려해서 질문과 답변등을 생성할 수 있어야 해"+
        "그러기 위해 중요한 내용도 잘 추려서 담아내야 해"+
        "너가 잘 이해할 수 있도록 구조화 되어야 해"
        },
        {"role": "user", "content": script}
      ]
    )
    return completion.choices[0].message.content

In [12]:
script = """
여러분 경제 성적에 봄이 왔습니다.
안녕하세요? 저는 박봄입니다. 반갑습니다.
경제라는 과목이 허들이 높아요.
이것을 수능 과목으로 선택해야 돼, 말아야 돼? 
경제 공부하고 싶기는 한데 이것이 가능해, 불가능해? 하는 마음으로
이렇게 클릭 한 번 했을 학생들을 제가 꽉 부여잡겠습니다.
경제 과목만큼 유용한 과목이 없어요. 성인이 되잖아요.
성인이 되면 영어를 언제까지나 붙들고 있어.
역사도 체계적으로 공부하고 싶어진다? 거기에 또 하나가 경제입니다.
어쩌면 경제에 관심이 제일 많을 수도 있지.
물론 성인이 돼서 만나는 실물 경제는 우리가 교과서에서 배우는 경제와 
조금 다르기는 한데 성인이 돼서 각종 경제 현상들을 막 보다 보면
경제 과목에 대해서 진짜 체계적으로 공부하고 싶어져요.
경제 교과서 그리고 수능 특강 또 개념 원리 경제 교재를 딱 보잖아요.
그러면 대학교 1학년 또는 2학년 때 가서 배우는 경제학 원론 배우는 것이 아니지.
원론서에 있는 가장 기본적인 내용들이 다 들어가 있는 느낌이 들어.
그래서 진짜 경제를 체계적으로 공부할 수 있는 기회다.
너희들이 무거운 마음으로 이것을 해? 말아? 하는 마음으로 톡톡톡 두드렸다면
제가 콱 붙들도록 하겠습니다. 여러분 경제 성적에 더불어서 봄도 드리고
제가 올해 여름에 마라톤을 했어요. 마라톤 되게 좋다는 것이야.
그래서 자신은 없었지만 신청을 했지. 대회에 신청을 했어요.
물론 마라톤 42.19 아니고요. 10km입니다.
그런데 제가 10km 마라톤을 하고 느낀 것은 세상에 어떤 것이 1시간을 투자해서
이렇게 성취감을 느끼게 할 수 있는 것이 무엇이 있을까?
진짜 그만한 것이 없는 것 같아요. 그래서 제가 마라톤의 진짜
너무 매력에 빠지기는 했으나 제가 그래도 하프를 하거나 풀코스를 뛰거나
그럴 마음은 없어요, 다만 10kg인데. 제가 그때 처음 마라톤 대회를 나가면서
딱 두 가지를 결심하고 나갔었어. 처음이니까 내가 10km 내내 뛴다는 것이 말이 돼?
약간 이러면서. 그래서 제가 무엇을 생각했냐면 절대 멈추지는 않겠다.
천천히라도 계속 뛰겠다. 그리고 걷지 않겠다. 그 두 가지였어.
걷지 않겠다, 쉬지 않겠다. 꼭 그 두 가지. 그래서 천천히라도 계속 뛰겠다고
생각을 했습니다. 처음 출발할 때 내 옆을 막 씽씽 지나가.
막 나는 저 뒤에서 이러고 천천히 진짜 뛰고 있었거든.
그런데 나중에 제가 그 속도로 계속 가다가
어떨 때는 조금 더 속도를 높일 수 있게 돼요. 가다가 나보다 앞서 갔던 사람들이
막 서 있고 막 걸어가고 이러고 있는 것이야.
그래서 제가 첫 마라톤에서 여자 중간보다 조금 잘했어요.
1시간을 넘기긴 했지만 기록상 그러긴 했지만 너무 뿌듯한 것이야.
제가 그때 느꼈습니다. 여러분 경제 과목도 그래.
경제 과목도 너무 한 번에 잘하고 빨리하려고 하지 마. 그럴 필요 절대 없어요.
중요한 것은 무엇이야? 계속하는 것이야. 꾸준히 하는 것.
절대 그냥 놔 버리지 않는 것. 놔 버리지 않는다, 하나.
마라톤과 똑같아. 놔 버리지 않는다, 하나. 또 너무 빨리 가려고 하지 않는다.
그러면 제가 손잡고 마라톤을 뛰듯이 잘 데려가 줄게.
절대 포기하지 말고 1강 왔는데 무엇을 포기야, 그렇지? 절대 끝까지 함께 가자.
여러분 인생에서 경제 그래도 내가 체계적으로 한번 배웠네, 라고 하는
그런 소중한 경험을 만들어 드리도록 하겠습니다. 여러분 교재와 짝꿍이에요.
그래서 교재 딱 보면서 그리고 강의 보면서 하시면 되겠습니다.
여러분, 제가 OT에서도 말씀드리지만 저는 단원명
그리고 중단원명을 너무너무 강조하고 있습니다. 왜?
경제 수능 특강 보면 200페이지 넘어. 개념홀릭도 거의 200페이지 가까이 돼.
왜냐하면 문제도 많이 있거든. 여기서 고작 몇 문제 나는 것이야?
20 문제 나는 것이야. 그러면 20 문제 어디서 난다? 골고루 난다.
5개의 단원에 중단원 모두 합치면 15개입니다.
15개에서 골고루 출제되고 있습니다. 따라서 단원명
그리고 중단원명 속에서 이것 중단원에서 핵심 주인공은 누구지?
그것을 캐치해서 그 핵심 주인공, 핵심 개념 또는 평가 요소
위주로 공부를 하셔야 됩니다. 그래서 큰 그림으로 15개의 중단원을
모두 여러분의 머릿속에 체계를 잡고
거기서 주인공들을 하나하나 정복해 가는 것이야. 좋아요.
Ⅰ. 경제생활과 경제 문제라고 되어 있습니다.
1단원은 전반적으로 경제에 대한 가장 기본적인 내용들을 공부한다고
생각하시면 됩니다. 기본적인 용어들, 이런 것들을 여기서 정리하는 것인데요.
01. 이렇게 되어 있어. 희소성과 합리적 선택이라고 되어 있습니다.
희소성이 무엇이야? 무언가 적은 것이지, 그렇지?
그렇지, 자원의 희소성 이런 것 되게 많이 들었잖아.
인간의 욕구에 비해서 자원이 희소하니까. 우리 돈, 시간, 좋은 대학
몇 개 안 되니까 그러니까 어때? 선택의 문제에 봉착하게 되는 것이지.
그래서 희소하기 때문에 선택의 문제에 봉착합니다.
이 선택의 문제를 무엇이라고 말하냐 하면 경제 문제라고 말하는 것이야.
그래서 여기서 그 이야기를 계속할 것이에요. 희소하기 때문에
선택의 문제에 봉착하고 그것을 무엇이라고 한다? 경제 문제.
경제 문제를 해결하는 방식에 따라서 시장 경제 체제, 계획 경제 체제 들어봤지?
그것이야. 그것 여기서 할 것이야. 그리고 가계, 기업, 정부가
경제 활동의 주체들입니다. 그래서 이 경제 활동의 주체들의 경제 활동에 대해서
공부하는 것. 전반적으로 딱 들어보니까 어때? 무언가 기본적인 것 하는 것 같구먼.
맞아요. 본격적으로 경제는 2단원부터 시작됩니다.
2단원 내 미시 경제 들어는 봤을 것이야.
그리고 3단원부터 거시 경제 파트가 들어가게 되는 것이지.
그래서 경제학 원론서, 개론서에 가장 기본적인 내용들이 다 들어가 있다고
생각하면 돼. 그러면 여기 공부할 내용 오늘부터 시작해서 3시간 동안
이어질 것이거든. 왜냐하면 여기서 기본적인 용어들에 대해서 탄탄히 알고 있어야
뒤에 가고 뒤에 가고 계속 나오는 용어들이기 때문에 흔들리지 않아요.
그래서 정말 세세하게 자세하게 제대로 하도록 하겠습니다.
희소성과 합리적 선택이라고 하는 본격적인 01 파트의 이야기는
다음 시간에 할 것이고요. 이번 시간에는 경제에 대한 가장 기본적인 내용들을
이야기할 것이거든. 그러면 이것이 시험에 안 나오는 것은 아니야, 얘들아.
그래서 첫 번째, 경제 활동의 유형. 그다음에 두 번째, 경제 활동의 주체.
그다음에 경제 활동의 객체. 이 이야기를 이번 시간에 하도록 하겠습니다.
그러면 경제 활동의 유형에는 무엇이 있냐 하면 여러분 경제 활동합니다.
우리의 삶은 다 경제 활동이거든, 우리의 일상생활은.
여러분 버스 타요. 여러분 맛있는 것 사 먹어요. 다 경제 활동입니다.
무슨 경제 활동? 소비라고 하는 경제 활동.
그래서 경제 활동의 유형에는 소비를 비롯해서 생산, 분배, 소비를
경제 활동의 유형이라고 말합니다.
그다음에 경제 활동의 주체는 앞서서 봤는데 이것들이야.
여기는 소개만 하는 것이야. 가계가 있고 기업이 있고 정부가 있어, 라고
여기 01에서는 소개만 하고요. 03에서 구체적으로 가계, 기업, 정부의
경제 활동에 대해서 하나하나씩 들어갈 것이고. 그다음에 객체가 무엇이야?
경제 활동의 대상이거든요. 들어만 두세요.
생산물과 생산 요소가 경제 활동의 대상입니다.
이렇게 됐어. 이것이 오늘 공부할 내용이야.
그러면 이것들이 시험 문제 나오느냐? 기본 개념이니까
시험에 안 나오는 것이 아니야. 여러분, 지금 01 파트의 출제 후보들입니다.
후보들 이렇게 네 가지나 있어요. 후보가 엄청 많아.
그중에서 경제 활동의 유형 주체와 객체. 지난 3년간 수능에서 단 한 번도 빠짐없이
계속 출제가 되었고 6월 9월에도 거의 출제됐다는 것을 알 수가 있어요.
이번 시간에 배우는 이것. 별표 3개짜리. 3개면 출제가 너무 잘 되는 것,
별표 3개 할 것이고요. 그리고 이것을 경제 순환 모형이라고 하는 데서
또 파악할 수 있거든요. 시험 문제는 이것을 그냥 물어볼 수도 있고
경제 순환 모형 속에서 물어볼 수도 있습니다. 이것이 중요한 이유는
이것이 3단원에서 다시 등장해. 그래서 이것을 스스로 그려 보려는 노력까지
하셔야 되겠습니다. 오늘 이 이야기들 쭉 해 보도록 할게요.
제가 학교에서 수업할 때 이렇게 오늘 공부할 내용이야, 라고 안내하면
학생들 되게 좋아해요. 이런 것 배우는구나.
그리고 금방 머릿속으로 앞으로 공부할 내용들을 머릿속에 탁 이렇게
체계를 잡습니다. 이렇게 넘기면 이것이 없어지는 줄 알았다.
그래서 무엇부터요, 무엇부터? 경제 활동의 유형.
그리고 주체, 객체 이렇게 해 보도록 하겠습니다.
여러분, 9페이지 넘기시면 되겠어요. 9페이지 넘겨 볼게요.
무엇부터 할 것이냐면 경제 활동의 유형부터 할 것입니다.
경제 활동은 우리가 살아가는 데 필요한 것을 무엇 하는 것? 지금 방금 했네요.
생산 그다음에 분배, 소비. 너무 쉬운데 이것을 배우고 그러나요?
그럴 수 있을 것 같은데 여러분이 이것 너무 쉽다고 이제 말 안 나올 것이야, 아마.
생산은 무엇이겠어? 생산, 무엇 만드는 것, 제조하는 것. 그렇지?
무엇 만드는 것 같아. 분배는 나누는 것 같아? 나누는 것 같고 소비는요? 사는 것.
이러고 끝나겠어? 그렇지가 않겠지? 수능에 이런 것을 내겠어?
물어보도록 하겠습니다. 생산, 만드는 것 맞아.
자동차 만들고 만두 만들어서 판매하고 이런 것 다 생산 맞습니다.
그러면 짠. 이것은? 저예요. 진짜 나는 벤자민 버튼의 시간은 거꾸로 간다야.
갈수록 나아지지 않았어? 이것 무려 14년 전일 것이에요, 아마.
2010년. 14년 전이면 너희 몇 살 때야? 막 6살 때 막 그럴 때야.
그때 나 여기 이러고 있었어. 법과 정치라고 하는 해설 강의.
그때는 법과 정치라고 하는 과목이 있었거든. 해설 강의하고 있어.
해설 강의하고 있는 나. 공짜로 하지 않아, 얘들아.
해설 강의하고 있는 나, 무엇 하고 있는 것이야? 소비해?
분배해? 나 경제 활동하는 것이야. 그렇지, 생산하고 있는 것이지.
교육 서비스를 만들어 내는 것이야. 따라서 생산이라고 말할 때는 무언가 물건,
물질, 생산물, 재화만 만들어 내는 것이 아니라
이렇게 교육 서비스를 만들어 내는 것도 생산이고요.
의사가 진료 활동하는 것도 생산이고 우리 정국이, 그렇지? 무엇이야?
뉴욕에서 공연한 것도 생산이고 그리고 오징어 게임, 이런 것들을 만들어 내는 것도
생산이라고 하는 것이야. 무언가 하나 더 덧붙지 않아? 알고 있었어?
그러면 다행이고요. 생산, 분배, 소비는 지금 제가 쓴 것처럼 만드는 것,
나누는 것 그다음에 무언가 사는 것.
이렇게 단편적으로 배워서는 당연히 안 되겠지요. 수능 경제인데.
그래서 이것들 간에는 유기적인 연결이 되어 있습니다.
그래서 9페이지에 경제 활동의 연관성이라고 필기 공간이 살짝 있어요.
거기에다가 같이 쓰면서 해 보도록 하겠습니다.
생산 그다음에 분배. 엄청 많이 쓰일 용어들이에요.
그리고 3단원 가서도 많이 쓰일 경제 활동의 유형입니다.
계속계속 많이 쓰일 것이고요. 이것들이 상호 유기적으로 연결되어 있다고
하는 것인데 제가 지금 교육 서비스를 생산해 냈습니다. 그렇지요?
그러면 제가 공짜로 안 한다고 했지, 그렇지?
생산에 참여한 대가를 받는 것을 분배라고 말하는 것이야.
다시 분배란 무엇이라고요? 생산 활동에 참여한 대가를 받는 것이거든.
그래서 제가 교육 서비스를 제공한 대가로 임금을 받아요.
임금을 받는 것이고 그 임금을 가지고 무언가 사과 패드, 사과 시계 이런 것들을
소비하는 것이지. 사과도 사 먹고 아무튼 이렇게 하는 것입니다.
그래서 유기적인 연관성이 있다고 말하는 것인데
여기 분배에 대해서는 조금 더 부연 설명을 할 필요가 있어요.
그래서 생산 활동에 참여한 대가로 이렇게 임금을 받는 것.
이것이 바로 분배라고 말씀드렸잖아요. 봐 봐. 여기 기업이 있어요.
분배 밑에 써 볼게요. 기업이 있고요. 기업이 이윤을 창출하거나
또는 생산해 내기 위해서는 무엇이 필요하겠어요? 
여러분, 이것도 미리 알아 두자.
기업이라고 말할 때 기업은 우리가 말하는 S 기업
그다음에 저기 공장이 크게 있는 중소기업, 자동차를 만들어 내는 무슨 기업.
이런 데만 기업이 아니에요. 우리 주변에 있는 온갖 종류의 자영업체들 있잖아요.
학원, 만둣집, 김밥집, 무빙의 봉석이네 돈가스집 이런 것 다 무엇이다? 기업입니다.
그래서 봉석이네 돈가스집에서 돈가스를 생산해 내려면 무엇이 필요하겠어?
건물도 필요하고 노동력도 필요하고 맞지요? 무언가 기계도 필요하고
삶는 것도 필요하고 튀기는 기계도 필요하고 그렇지?
생산에 필요한 것을 무엇이라고 말하냐 하면 생산 요소라고 말합니다.
그래서 생산에 필요한 것. 봉석이네 돈가스집에서 돈가스를 생산하려면
일단 노동이 필요해요. 그래서 이 노동이 대표적인 생산 요소거든요. 드러나야 돼.
조금 이따 또 할 것이야. 또? 토지가 필요하지. 그다음에요? 자본이 필요합니다.
자본, 토지가 무엇인지 조금 더 자세히 설명할 것이에요. 이것들이 필요합니다.
그래서 이것들을 어디로부터 받아 와요? 가게로부터 받아 오게 됩니다.
생산 요소를 가게로부터 제공받게 돼요.
그러면 가계가 이런 생산 요소를 제공한 대가를 받게 되지, 그렇지?
앞서 여기 봤다시피 교육 서비스라고 하는 노동을 제공한 대가로
무엇을 받는 것이야? 임금. 토지를 제공한 대가로 무엇이지? 지대를 받게 돼요.
조금 이따 한 번 더 정리할 것이에요. 자본을 제공한 대가로 무엇을 받게 돼?
이자를 받게 됩니다. 이것들을 무엇이라고 부르는 것이야? 이것이 바로 분배입니다.
오케이. 그래서 분배받은 것은 가계 입장에서 무엇이 돼?
임금, 지대, 이자를 받으면 가계 입장에서 소득이 됩니다.
그리고 이 소득을 기반으로 해서 소비하게 되는 것이야. 오케이?
여기까지 한 것 다시 한번 정리해 볼게요.
생산, 분배, 소비가 유기적으로 연결되어 있다. 그렇지?
생산에는 재화뿐만 아니라 서비스도 생산하는 것이야.
교육 서비스, 의료 서비스. 그래서 생산 활동에 참여한 대가,
생산 요소를 제공한 대가로 이런 것들을 분배받게 되는 것이고
생산 활동에 참여한 대가가 그래서 무엇이야? 분배야, 분배.
분배는 가계 입장에서 소득이 되고 이것이 소비의 원천이 되는 것이지요.
다 했어, 다 한 것이야. 하나로 정리해 보도록 하겠습니다.
9페이지 아래에 여기서 조금 또 더 들어갑니다. 이것들을 계속 물어보는 것이지.
시험에서. 그래서 문제들까지 적용해 볼 것이에요. 경제 활동의 유형.
재화나 서비스를 만들어 내거나. 재화, 자동차를 만들어 내.
또 서비스를 만들어 내거나 이미 만들어진 재화나 서비스의 가치를 증대시키는 활동.
그래서 자동차를 만드는 것도 생산이고요. 자동차를 만들어서 홍보해.
그다음에 판매해. 이미 만들어진 재화나 서비스의 가치를 올려 주고 있는 것이지,
그렇지? 이런 것들도 다 무엇이 된다? 그렇지요. 생산이 됩니다. 홍보도 생산이고요.
판매도 생산입니다. 이해돼? 그래서 생산을 무엇이라고 말하냐 하면 이렇게 말합니다.
부가 가치의 창출이다. 오늘 이번 시간에 하는 것은 자잘한 개념들이 엄청 많아.
생산, 분배, 소비, 생산 요소, 임금, 지대 자잘한 것들이 되게 많지요?
그것들을 여러분이 체계적으로 머릿속에 딱 정리를 해 두셔야 됩니다.
이렇게 펼쳤다가 다시 정리했다가 계속 반복할 것이거든?
부가 가치를 창출하는 것은 생산이라고 하는데 부가에서 가치가 생기는 것이지.
여러분이 만둣집을 하면 만두 재료를 사서 만두를 만들어서 이윤을 붙여서 팔잖아요.
부가 가치의 창출. 새롭게 더해진 가치를 창출하는 것을 생산이라고 말합니다.
따라서 재화를 제조하는 것뿐만 아니라 운송도 저장도 판매도 다 무엇이다?
다 생산입니다. 운송, 저장, 판매, 홍보, 수리 이런 것 다 생산이라고 하는 것이야.
생산 하면 자동차 생산, 이렇게만 생각하면 안 된다는 것이야.
제조물, 무언가 재화를 제조하는 것뿐만 아니라 엄청 이렇게
부가 가치의 창출. 이런 것들이 다 키워드입니다. 이런 것들을 다 물어본다는 것이지.
두 번째, 분배는 생산 활동에 참여해서 그 대가를 받는 것이에요.
그래서 생산 활동에 참여한다고 할 때는 노동을 제공하거나 토지를 제공하거나
자본을 제공하는 것. 그래서 그 대가로 노동을 제공하면 무엇이라고 했어?
임금, 지대, 이자. 그래서 이런 것들을 받으면 이것이 무엇이 된다고?
가계의 소득이 된다. 앞서 딱 정리했지요, 그렇지요?
이것들 다 선지 하나하나가 다 될 수 있어요.
거의 3년간 100% 수능에 출제했던 것. 소비는 무엇이야? 구매하는 것.
무엇 하기 위해서? 만족을 얻기 위해서. 이것 2개가 키워드입니다.
사면 기분 좋잖아. 그래서 열심히 벌어서 또 열심히 사잖아요. 왜? 기분 좋으니까.
그래서 소비의 키워드는 사는 것. 이건 너무나 당연한 것이고요.
만족을 얻기 위해서예요. 만 족은 다른 말로 효용이라고 말하기도 하고
편익이라고 말하기도 합니다. 이것의 키워드야.
그러면 또 하나, 여기 보면 이렇게 제가 넓게 해 놨어. 거기다 무엇을 쓰라는 것이지.
봐요. 제가 사과를 구입했습니다. 그래서 아침마다 먹어요. 왜?
너무 건강에도 좋고 피부에도 좋다길래 제가 이렇게 먹습니다.
그리고 제 딸이 캠퍼스가 넓은 좋은 대학교에 합격을 해서
캠퍼스를 다니려면 다리가 너무 아프다고 그러길래 대학교에서 스쿠터 타고 다녀.
통학용으로 사 주었습니다. 스쿠터 구입했어요. 됐어, 이것 다 무엇이야?
그렇지, 이것 다 무엇이냐 하면 사과를 구입했고 스쿠터를 구입했습니다.
만족을 얻기 위해서 무언가를 뭔가를 구입하는 활동이니까 이것들 다 무엇이겠어?
이것은 소비야. 여러분, 이 아래에 이렇게 쓰시면 됩니다. 이것 소비예요.
그런데 잼 가게 사장님이 사과잼 만드는 재료로 사과를 구입했습니다.
그다음에 어떤 음식점 사장님이 배달용 오토바이가 자꾸 고장이 나서
배달용 스쿠터를 구입했어. 이렇게 했어. 봐 봐. 구입했어, 구입했어.
여기도 구입했고 구입했고 이것은 소비, 이것은? 그렇지, 소비일 리가 없겠지.
이것도 시험에 출제된다. 이것은 소비가 아닙니다.
소비의 키워드 무엇이라고 했어요? 만족을 위해서.
만족, 편익, 효용을 극대화하기 위해서 구매하는 것을 소비라고 하고요.
여기는 잼 가게, 음식점 이것들 다 무엇이야? 이것들 다 기업입니다.
기업은 대기업, 중소기업 공장 있고 이런 것만 기업 아니라고 분명히 말씀드렸습니다.
기업이 재료, 배달용. 무언가 재료나 생산 요소로써 구입하는 것이야.
그러면 이것들은 무엇이 되냐? 이것들은 생산이 되는 것이야.
별것 아닌 개념인데 엄청나지? 이런 것 물어본다니까.
잼 가게 사장님이 재료를 구입하기 위해서 사과를 구입했다.
재료로써 사과를 구입했다, 소비. 이렇게 해 놓으면 그렇지요. 틀리는 겁니다.
다시 하겠습니다. 여기까지 한 내용 쭉. 제가 왜 다시 하는 줄 알아요?
자잘한 것이 너무 많다니까. 여러분이 머릿속으로 생각하면서 여러분 것으로
정리해야 된다니까. 나만 다시 하면 안 돼. 멈춤하고 여러분이 다시 해.
경제 활동의 유형의 생산, 분배, 소비가 있는데 생산은 재화뿐만 아니라
제가 하는 서비스를 제공하는 것도 만드는 것도 생산이다.
운송, 저장, 판매 이런 것들 다 포함해서 부가 가치를 창출한다는 표현을 써요.
생산 활동에 참여한 대가를 받는 것이에요. 이런 것, 이런 것. 이것이 분배다.
소비, 사는 것. 만족을 얻기 위해 사는 것입니다. 이런 것들은 무엇이라고 불렀어?
그렇지. 안 잊어버렸을 것이야. 체크를 해 보도록 하겠습니다.
한번 여기 집어넣어 볼까? 어디 해 볼 것이냐 하면 12페이지로 가 보도록 하겠습니다.
생산, 분배, 소비 활동의 구분이라고 되어 있고요.
제가 BTS 팬으로서 BTS 멤버들로 해서 써 봤습니다.
BTS가 CBS 제임스 코든쇼에 출연하여 노래를 부른다. 무엇일까요?
그렇지, 엄청난 서비스를 제공하고 있는 것이잖아. 그래서 이것 무엇이다?
생산이라고 해야 합니다. 그래서 가수가 노래 부르는 것.
제가 이렇게 인터넷 강의 찍는 것, 다 생산. 윤기가 한강 둔치에 피자를 배달하였다.
가수 되기 전에 배달 알바하고 그러셨대요, 윤기 씨가.
슈가 씨가 배달하였다. 배달이 무엇이야? 배달, 그렇지요.
운송, 저장, 판매 이것 다 생산입니다.
은근 헷갈려. 여러분이 먼저 해 봐야 돼. 생산.
남준 씨, 미술 전시회 가는 것 되게 좋아해서 해 봤어. 책을 구입했다.
왜? 읽으면 되게 기쁘겠지. 이 사람 기쁠 것 아니야, 소비.
그다음에 지민이네 어머니는 정기 예금에 대해서 이자를 지급받았다.
예금이 무엇이에요? 자본을 제공한 것이에요.
어머니가 은행에 돈을 저금하면 은행은 기업이나 어디에 빌려줄 것이야.
그리고 이자를 분배받는 것입니다. 그래서 이것 무엇이다? 분배.
이렇게 하시면 되겠습니다. 태형이 편의점에서 아르바이트를 하고
시간당 노동력을 제공하고 임금을 받은 것이지요. 그렇지, 분배야 분배.
정국이는 농촌에서 배추를 도시로 운송이라고 그대로 나왔지? 그렇지요, 생산입니다.
그다음에 빵집 주인 석진이가 요리 잘하시지요, 진이 씨가 요리 잘하셔.
요리 좋아하셔서 식빵을 만들기 위해, EBS 강사 봄이가 수제비를 만들기 위해서
빵집 주인 그다음에 나 그래서 구입, 똑같이 밀가루 구입했습니다.
그러나 다르지요. 그렇지, 진이 씨가 하면 이것은 무엇이다? 생산입니다.
그리고 제가 하는 것은요? 남편과 맛있게 끓여 먹어서 기쁘려고 그래서 무엇이지?
소비지, 소비. 이렇게 해야 되는 것이야. 잘했습니다. 지금까지 어떤 이야기했어요?
경제 활동의 유형에 대해서 했습니다. 생산, 분배, 소비 별것 아닌 것을 이렇게 했어.
중요하니까. 시험 문제 나니까 하는 것이지. 좋아요.
두 번째, 경제 활동의 주체보다 객체를 먼저 하겠습니다.
왜냐하면 객체 이야기를 이미 했거든. 객체 이야기를 했습니다.
여기서 보시면 나와 있어요. 무엇이냐, 책을 샀다.
이것이 대상이지. 소비 활동의 대상. 밀가루를 샀다, 소비 활동의 대상. 그렇지요?
그다음에 노래를 그러면 우리는 듣지, 노래.
이것이 다 경제 활동의 대상들이 막 나와 있어요.
그래서 경제 활동의 대상은 크게 생산물과 생산 요소가 있다. 용어들입니다.
너무 자잘하게 많아서 제가 계속 이야기하는 것이야. 생산물과 무엇이 있다?
생산 요소가 있다. 생산물에는 만족감을 주는 형태가 있는 물질, 재화입니다.
그런데 여러분, 이렇게 물어보면 헷갈린다? 교육이 생산물이냐?
그러면 교육이 무슨 생산물이에요? 물 하면 물질로 유형이 있어야 될 것만 같잖아.
아니라는 것이야. 여기 보면 더 나은 삶을 위해 필요한 무형의 인간 활동.
그래서 예술 공연, BTS 공연도 생산물이야. 에? 하면 안 돼. BTS 공연도 생산물이고
제 인터넷 강의도 생산물이야. 즉, 서비스도 생산물입니다.
그래서 이것 별것 아닌 것 같은데 나중에 틀려. 예술 공연이 생산물이다.
그러면 생산물 아닌 것 같아, 어쩐지. 그것은 그냥 우리의 고정 관념이지요? 
개념적으로 설명하면 무형의 인간 활동과 유형의 물질을 다 합쳐서
생산물이라고 합니다. 꼭 기억해 놔야 되지요, 그렇지요?
두 번째, 생산 요소. 앞서도 말했어요. 생산 요소는 생산 활동에 필요한 것.
봉석이네 돈가스집을 운영하려면 일단 노동력이 필요합니다.
그래서 필요한 것을 얻기 위해 인간의 활동, 노동이에요. 생산 요소 노동입니다.
그런데 노동에서 중요한 것 또 하나 있어, 체크해야 될 것. 이것.
정신적 활동도 노동이다. 노동 하면 어쩐지 육체적인 것만 노동인 것 같아.
그런데 정신적인 것도 노동입니다. 그래서 저 노동자예요.
노동을 제공하고 임금을 받거든. 그래서 제가 막 이렇게 힘써서 하지는 않지만
정신적인 활동도 노동이다. 제가 대학교 때 정치외교학과 나왔거든요.
술자리를 가지면 이런 이야기들을 하곤 했어.
그런데 친구 한 명이 이러는 것이야. 술을 막 먹더니 대학교 때예요.
그 아이가 "난 노동자가 되지 않을 것이야." 막 이랬어요.
그래서 우리가 "네가 노동자가 되지, 무엇이 된다는 것이야?" 이러면서
그런 이야기를 했었어요. 이해 안 되지? 왜 그런 이야기들을 했는지.
그런데 아무튼 그 아이 딴은 아마 그랬던 것 같아.
노동자는 무언가 육체적으로 되게 힘들고 막. 그런데 노동자가 되지 않겠다고 했던
그 아이 말은 '나는 부자가 될 것이야.' 그런 말을 하려고 했던 것 같아.
정치학과씩이나 돼서. 그런데 그 아이 나중에 "그 아이 노동자로 살아요.". 
노동을 제공하고 임금을 받는 노동자로 살고 그다음에 한참 뒤에 우리가
다 직장인이 돼서 만났을 때 그때 그 이야기를 하면서 그 아이가 노동조합 가입해서
활동하고 있거든. 그래서 그 이야기하면서 막 웃었던 그런 생각이 납니다.
그래서 노동은 무엇이다? 정신적인 활동도 노동이라고 하는 것.
그다음에 봉석이네 돈가스 만들려면 땅이 있어야 건물도 짓고 이럴 것 아니야.
그렇지, 토지가 필요합니다. 그런데 제가 지금 토지라고 말하면서
땅이라고 말하긴 했어. 땅이 토지가 맞긴 맞아.
하지만 생산 요소에서 토지라고 말할 때는요.
자연으로부터 획득한 자원은 모두 토지라고 부르기로 했습니다.
그래서 하천, 바다, 산림, 지하자원, 수산 자원 이것도 다 토지라고
부르기로 한 것이야. 다시 말하면 자연으로부터 얻은 것은
다 토지라고 부르기로 한 것이야. 생산 활동에 필요한 것 중에
사람이 일단 필요하지. 자연으로부터 무언가 필요하지, 이런 것이. 생산 활동하려면.
이것들은 다 토지라고 부르기로 했고요. 인간이 만들어 낸 생산 요소도 있습니다.
무엇이겠어? 공장, 기계 이런 것 필요하지, 그렇지?
그것들은 다 무엇이라고 부르기로 했냐 하면 자본이라고 부르기로 했습니다.
우리는 자본 하면 화폐라고 생각하잖아. 토지 하면 땅이라고 생각하고.
그런데 그렇지 않다고. 생산 요소, 생산 활동에 필요한 것이 인간의 몸과 토지. 
자연으로부터 얻은 것과 인간이 인위적으로 만들어 낸 도구, 기계, 공장
이런 것들이 필요한데 자연으로부터 얻은 것은 다 토지라고 하기로 하고
인간이 만들어 낸 물적 요소. 공장, 기계, 도구 이런 것들은 다 자본이라고
부르기로 했다. 오케이? 조금 들어가니까 무엇이 조금 더 있고 조금 더 있고 이렇지?
짜증 나게 하지, 그렇지? 되게 간단해 보이는 용어들인데. 이것까지 정리해야 됩니다.
수능 특강이 나중에 기본 문제에서 이런 것 물어봐.
산림, 수산 자원은 토지다. 그러면 무슨 헛소리야? 이렇게 하면 안 된다고.
그래서 이런 경제 활동의 대상을 객체라고 하는 것이고요.
재화와 서비스는 생산물 시장에서 거래가 되고요.
그다음에 노동, 토지, 자본은요. 생산 요소 시장에서 거래가 됩니다.
그냥 써 봤어요. 여기 쓸 필요는 없어요. 시장에서 거래가 됩니다. 됐지요?
그래서 해 봤고요. 이제 경제 활동의 무엇 할 것이다?
주체에 대한 이야기를 해 보도록 하겠습니다. 그 위에 있어요.
이것부터 먼저 했어. 경제 활동의 주체는 누구야? 가계, 기업, 정부다.
그리고 해외 부분까지 있어. 민간 부문을 바로 가계, 기업이라고 말하고요.
그래서 다음은 민간 부문의 경제 활동 주체에 대한 설명이라고 하면
정부 빼놓고 가계와 기업만 생각하면 되는 것이야.
공공 부문까지 들어가면 정부가 들어가는 것이에요.
다 밑줄이 되어 있네? 그런데 이것이 있다. 편익의 극대화, 이익의 극대화.
이것이 완전 중요한 말이다. 여기도 다 외워야 될 것들입니다.
경제 활동의 주체로서 민간 부분은 가계와 기업이 있습니다.
가계와 기업 중에서 편익의 극대화를 추구하는 경제 활동의 주체는?
편익, 같은 말로 무엇이야? 만족 또는 효용. 어떻게 쓰셔도 됩니다.
극대화를 추구하는 주체는 가계입니다.
왜냐하면 가계가 소비 활동의 주체이기 때문입니다. 제가 아까 말씀드렸지요.
소비는 무엇 하는 것? 구입하는 것. 키워드는 그런데 무엇이다?
구입보다 그 앞에 있는 만족을 위해서 구입하는 것이라고 말씀드렸잖아요.
따라서 가계는 소비의 주체이고요. 만족, 효용, 편익 극대화를 추구합니다.
이것 맨날 선지로 나와요. 확인할 수 있어요. 그다음에 이익 극대화
또는 이윤 극대화를 추구한다, 이렇게 말합니다. 누구겠어요? 그렇지요, 기업입니다.
기업은 무엇의 주체야? 경제 활동 중에서 생산의 주체입니다.
그래서 가계는 소비의 주체로서 효용, 편익, 만족 극대화
그리고 기업은 생산의 주체로서 이윤 극대화를 추구한다. 되었습니다.
다 외울 것이지? 좋아, 우리 앞에서 이미 이것을 했네.
경제 활동의 객체로서 생산물 그리고 생산 요소가 있다고 했지요.
생산물은 무엇이지? 재화, 서비스. 생산 요소는 무엇이라고 했어?
노동, 토지, 자본이야. 이제 보겠습니다. 가계는 생산물
재화와 서비스를 무엇 하겠어? 그렇지, 사.
그래서 생산물 시장의 수요자다. 이렇게 이야기합니다. 다 외워야 합니다.
그러나 이것 억지로 외우지는 않지, 여러분. 그냥 우리 일상생활이 이렇잖아요.
그리고 가계는 노동을 제공하고. 제일 앞에 그림 그렸어. 토지, 자본을 제공합니다.
그래서 생산 요소 시장의 공급자입니다. 그다음에 기업을 보도록 하겠습니다.
기업은 어때요? 기업은 재화와 서비스를 공급하는 것이야.
기업이 생산 활동을 하려면 무엇이 필요해? 노동 사 와야지, 토지 사 와야지.
그렇지, 사 와야 돼. 생산 요소 시장의 수요자가 됩니다.
이제 공공 부문 들어가면 누가 들어가? 정부가 들어가게 되고요.
정부, 경제 활동의 주체입니다. 경제 활동의 주체로서 정부는 사회적
후생 극대화를 추구한다, 이렇게 말합니다.
후 자가 뭐냐 하면 두터울 후 자야. 생은 생활 그 생 자.
우리가 알고 있는 것. 그래서 생을 두텁게 한다.
쉽게 말하면 복지 이런 것이라고 생각하시면 됩니다.
정부가 경제 활동하는 것은 03 파트에서 할 것인데
정부의 경제 활동은 재정 활동이거나 이런 것이거든요.
그 일을 통해서 사람들의 삶을 두텁고 따뜻하게 하려는 것이겠지.
그래서 그 표현을 후생 극대화다, 이렇게 말하는 것이에요. 무엇의 주체?
정부가 나라 살림하는 것을 재정 활동이라고 하거든?
이것 03에서 다시 할 것이야. 이것들 다시 다 해. 그래서 재정의 주체다.
이렇게 말하는 것이고요. 정부가 재정, 다시 말하면 복지 제도도 운영하고
교육 제도 이런 것 운영하려면 교육 예산도 보내고 하려면 생산물.
컴퓨터도 필요하고 책상도 필요하고 이러지 않겠어? 그렇지요,
이것은 사 와야 됩니다. 수요자. 정부도 정부 사업하고 하려면 건물도 필요하고
그렇지 않아? 그렇지, 노동력도 필요하고 생산 요소 시장도 수요자입니다.
지금 이 빈칸들이 다 무엇 한다? 다 시험에 나오고 있습니다.
여기에 하나 덧붙여 갑니다. 외국도 같이 경제 활동을 하게 되면 외국과는 무엇 해?
무역하지. 4단원에서 공부할 것입니다.
그래서 외국은 수출입의 주체다, 이렇게 말합니다.
자국의 이익 극대화를 위해서. 이렇게 했습니다. 경제 활동의 유형 살펴봤어요.
자잘한 것들을 다 기억하고 있는지 반드시 멈춤하고 복습해야 돼.
두 번째, 경제 활동의 객체 봤습니다. 대상이에요.
서비스, 예술 활동 이런 것도 생산물이다.
토지가 땅만 해당되는 것 아니라고 했어, 그렇지?
그다음에 무엇이지? 자본이 돈만은 아니야. 화폐만 자본이 아닌 것이야. 그렇지요?
그리고 경제 활동의 주체 가계, 기업, 정부고요.
가계와 기업을 민간 부문이라고 한다 했습니다. 이것을 하나로 딱 그려요.
그러면 이것이 무엇이 되냐 하면 경제 순환 모형이 됩니다. 너무 중요해서요.
이것이 여기만 나오지를 않아요. 3단원에서 아주아주 중요하게 다뤄지는
국민소득 3면 등가의 법칙이라고 있는데 그 3단원에서도 엄청 중요하게 다뤄지는
따라서 이번 시간에 이것 공부하고 나면 여러분들은 이것을 스스로 그려 보려고
하셔야 됩니다. 스스로 그려 봐. 봤던 것 있네. 생산물 시장과 하나.
그러면 무엇이겠어, 이것이? 그렇지, 생산 요소 시장이 있습니다. 그렇지요?
생산물 시장과 생산 요소 시장이 있네. 생산물 무엇이라고 했어요?
여기 있네. 재화와 서비스. 이 하나의 경제 순환 모형 안에
경제 활동의 대상, 경제 활동의 주체, 경제 활동의 유형을
하나의 그림 안에 다 표현할 수가 있어요. 그래서 이것이 엑기스야.
이것을 스스로 그려 볼 수 있어야 된다고 말하는 것이야.
재화와 서비스가 거래되는 시장이 생산물 시장.
생산 요소 시장에서 무엇이 거래되겠어? 맞아. 노동, 토지, 자본입니다.
실물의 흐름과 화폐 흐름이라고 나와 있어요.
실물의 흐름과 화폐의 흐름은 어떻게 되겠어? 반대로 가겠지, 맞지요?
우리가 어떤 햄버거를 이렇게 받았어요, 재화.
그러면 반대로 화폐가 이렇게 움직여야 되겠지? 그것이야.
그래서 실물의 흐름과 재화의 흐름은 반대로 간다는 것이지.
그러면 실물의 흐름을 보겠습니다.
생산물 시장으로 실물을 즉, 재화 서비스를 보내고 있어.
즉, 재화 서비스. 생산물의 공급자야. 그러면 이것이 무엇이 되겠어요?
맞아, 기업이야. 그래서 이렇게 가. 그러면 재화 서비스를 이렇게 가져오고
이렇게 사 갑니다. 반대로 화폐가 이렇게 나가고 있지요?
그러면 재화 서비스의 수요자, 누가 돼요? 그렇지, 가계지.
그래서 여러분, 잘 봐. 재화 서비스를 기업이 이렇게 만들어 내고 있습니다.
생산물을 만들어 내는 것을 무엇이라고 했어?
그래서 여기서 생산 활동이 나타나는 것이야.
그래서 생산한 재화 서비스를 가계가 이렇게 가지고 오며
반대로 이렇게 사 가는 것. 여기서 소비가 나타나는 것이지요. 그렇지요?
이것이 구입하는 것이니까. 하나의 그림 안에 다 표현이 된다니까.
이제 여기 반대로 보겠습니다. 여기 보시면 노동, 토지, 자본이야.
노동, 토지, 자본은 가게가 이렇게 주는 것이에요.
가계가 생산 요소 시장의 무엇이다? 공급자라고 말했었잖아.
이것 줬지. 그러면 이렇게 기업이 가져가지요. 그러면 대가를 줘야 될 것 아니야?
생산 요소를 제공한 대가, 그것이 바로 무엇이라 그랬어? 계속 나와.
임금, 지대, 이자. 이것들을 이렇게 무엇 하고 있어? 분배받고 있는 것이지.
그래서 분배가 나타나. 그러면 이것은 가계 입장에서 무엇이 돼? 소득이 돼.
돌고 돌고 돌고 돌고 그래서 순환 모형이 되는 것이에요.
하나의 그림 안에 생산, 분배, 소비, 가계, 기업, 정부, 생산물, 생산 요소
다 나타났습니다. 이것 설명하려고 여태 이렇게 온 것이야.
이것 그러니까 여러분이 그릴 수 있어야 돼. 문제에 적용시켜 보도록 하겠습니다.
몇 페이지냐? 아직 교재가 안 나왔어요. 12페이지입니다. 여러분 한번 해 보자.
국민 경제 순환 모형이라고 해 놓고 무엇이야? 그렇지.
실물의 흐름이라고 되어 있습니다. 실물이야, 실물.
화폐의 흐름은 아니고 실물의 흐름. 그런데 다 빈칸이야.
그런데 여기서 무엇이라고 했냐 하면 민간 부문이라고 했으니까
A, B는 무엇 아니면 무엇이겠어? 가계 아니면 기업이겠지. 그렇지? 해 보겠습니다.
(가) 시장이 생산물 시장이라면 여기 생산물이에요. 실물의 흐름이야.
생산물을 보내 주고 있네요. 그러면 여기 무엇이 돼요? 그렇지, 기업.
그러면 A는 기업이다. (가) 시장이 만약에 생산 요소 시장이라면
(나) 시장이 무슨 시장이야? 생산물. 그러면 여러분, ㉠이 무엇이겠어?
실물의 흐름이라며 실물. 생산물, 그렇지. 재화와 서비스다.
여러분, 그냥 따라 그리면 안 돼요. 여러분이 멈춤하고 여러분이 해 봐야 돼요.
그냥 따라 쓰고 있으면 아무 의미. 멍 이해되네. 끝, 이러면 안 돼.
진짜 공부는 생각하는 것. (나) 시장이 생산물 시장이라면 B는 무엇이겠어?
생산물을 제공합니다. 그렇지, 이번에는 B가 기업이야. 그러니까 여기도 기업.
다시 갑니다. 이제 이것도 나왔네요. 이것 외우라고 했지.
효용 극대화를 추구하는 경제 주체라면 A는 무엇이겠어? 맞아요, 맞아요.
가계입니다. 그렇다면 (나) 시장은 어떤 시장일까요?
이 시장에서 가계로 이렇게 들어가. 가계가 무엇이야? 수요자야.
그렇지, 생산물이지. B가 이윤 극대화를 추구한다면 B가 무엇이겠어요? 기업입니다.
그러면 A는 (가) 시장의 그대로네. 여기 생산물 시장이고 가계고 여기 생산 요소.
그렇지, 여기 생산 요소가 되겠지요. 나만 아는 것이면 진짜 안 되는데.
여러분이 지금 이 내용들을 다 기억하고 있어야만 할 수 있는 것들입니다.
그래서 비슷한 문제가 여기 있어요.
이 문제는 여러분이 풀어 보실 수 있을 것 같고요. 여러분, 한번 풀어 보세요.
맨 마지막에 있는 4번 문제이고요. 여러분, 2번 문제 보고 마무리하도록 하겠습니다.
2번 문제, 경제 활동의 유형. 이렇게 단독으로 나오기도 해.
3월에 나왔던 문제입니다. 분배와 생산이다, 이렇게 되어 있습니다.
경제 활동 유형 가운데서 분배와 생산 중에 하나라고 되어 있어요.
사장님, 치킨 배달하겠습니다. 아까 윤기 씨가 했는데 무엇이에요?
생산이지. 그렇지, 생산이고요. 월급을 입금하셨군. 임금을 분배받은 것이지요.
분배입니다. ㄱ. 효용을 얻기 위한 것이다. 누구의 키워드라고 했지요, 제가?
그렇지, 소비가 만족을 얻기 위해서 구입하는 것이라고 했습니다.
여기 소비는 안 나왔어요. 그러니까 틀렸지.
두 번째, 환자가 의사의 진료를 받는 행위. 의사의 진료를 받는 것이야.
의사의 진료를 사는 것이지. 그렇지, 이것은 무엇이야? 소비야, 소비.
헷갈리면 안 돼. 의사가 진료 행위를 하는 것은 생산인데 받는 것이니까 무엇이다?
소비다. 그다음에 (나)는 (가)의 과정, 생산에 참여한 대가를 받는 것이니까 맞고요.
여러분, (가) 생산은 부가 가치 창출. 계속 말씀드렸던 내용입니다. 맞지요?
다 했습니다. 여러분 경제에 봄이 올 것 같습니까? 꼭 그러셨으면 좋겠고요.
오늘 1시간 동안 생산, 분배, 소비, 경제 활동의 주체, 객체 등
기본적인 개념들을 했고 자잘한 개념들이 많으니까 어떻게 해야 된다?
계속 반복해서 복습해야 됩니다. 다음 시간에 본격적으로 희소성에 대한
이야기를 하도록 하겠습니다. 우리 다음 시간에 만나도록 하겠습니다. 안녕?
"""

teacher = {
      "나이":"25살",
      "이름":"채송화",
      "성별":"여성",
      "직업":"고등학교 사회선생님",
      "성격":"온화, 다정, 나긋",
}

student = {
      "나이":"초등학교 4학년",
      "이름":"짱구",
      "성별":"남성",
      "직업":"학생",
      "성격":["개구장이", "발랄"],
}

qeustion = "학습한 개념을 잘 이해했는지 스스로 확인하기 위해 자신의 일상과 연관지어 예를 들고 이것이 맞는지 확인하는 질문"
qeustion_num = 5

generator = Q_A_Generator()



In [ ]:
script_summary = generator.summary(script)

In [3]:
print(f"len(script) = {len(script)}")
print(f"len(script_summary) = {len(script_summary)}")

len(script) = 18456
len(script_summary) = 828


In [19]:

generator.make_q_a(script, teacher, student, qeustion, qeustion_num)

{'담화1': {'질문': "안녕하세요, 짱구야. 오늘 내가 배운 거 짱구에게 들려주고 싶어. 경제 수업에서 배운 경제활동의 유형이랑 주체에 대해 알게 되었는데, 일상 생활 속에서 예를 들어 보려고 해. 우리 가족이 마트에서 장을 보고 그걸 집에 와서 맛있는 저녁을 만들어 먹는 것, 이게 바로 '소비' 활동이 맞을까?",
  '답변': "응응, 채송화 선생님! 그건 맞아요. 마트에서 장보는 거랑 집에서 요리해서 먹는 건 우리가 뭔가를 사서 만족을 얻으려고 하는 거니까 '소비' 활동이 맞아요!",
  '대답': '맞아, 짱구! 정확하게 이해했네. 우리의 일상 속에서 무엇인가를 구매해서 사용하고 만족을 얻는 행위가 바로 소비활동이야. 잘 이해했어!'},
 '담화2': {'질문': "짱구야, 또 하나 궁금한 게 있어. 내가 강아지 간식을 만들어서 동네 사람들에게 팔기 시작했어. 그럼 이걸 '생산' 활동이라고 볼 수 있을까?",
  '답변': "음, 그걸 팔아요? 그럼 채송화 선생님이 만든 간식은 다른 사람들이 살 수 있는 거니까, 물건을 만들어서 판매하는 거잖아요? 그러니까 '생산' 활동이 맞는 거 같아요!",
  '대답': '정말 잘 이해했어, 짱구! 내가 간식을 만드는 과정부터 판매하는 것까지 모두가 생산 활동의 일부분이야. 이런 활동을 통해 무언가를 만들어내고, 그걸 시장에 공급하는 과정이거든.'},
 '담화3': {'질문': "지금부터 조금 어려운 이야기를 해볼게, 짱구. 선생님이 주식 투자를 해서 이익을 얻었다고 가정해 보자. 그리고 이 이익으로 새로운 노트북을 구매했어. 이런 경우, 내가 주식에서 이익을 얻는 것도 '분배'에 해당할까?",
  '답변': "어, 주식에서 돈을 벌었다고요? 그러면 선생님이 무언가 활동에 참여해서 그 대가로 돈을 받은 거잖아요. 어렵지만, 그게 선생님의 활동에 대한 '분배'로 받은 거라고 생각해요!",
  '대답': "맞아, 짱구. 정말 잘 따라왔어. 주식 투자를 통해 얻은 이익도 나의 자본을 제공하고 그 대가로 받은 것이기 

In [20]:
generator.make_q_a(script_summary, teacher, student, qeustion, qeustion_num)

{'담화1': {'질문': '채송화 선생님, 저번에 배운 경제 활동에 관한 것 맞나요? 제가 어제 가족과 함께 마트에 갔었는데요, 우리가 장을 보면서 장바구니에 물건을 담는건 ‘소비’ 활동이라고 생각해도 되죠?',
  '답변': '맞아, 짱구야. 네가 마트에서 물건을 고르고 구매하는 그 행위는 바로 소비에 해당해. 사람들이 물건을 사서 자신이나 가족의 만족도를 높이는 과정이지. 잘 이해하고 있는거 같네.'},
 '담화2': {'질문': '그렇다면, 아빠가 일하러 회사에 가시는 건 기업에서의 ‘생산’ 활동에 참여하는 거로 볼 수 있나요? 아빠는 밥 벌어오신다고 말씀하시던데 맞죠?',
  '답변': '네, 정확해 짱구야. 네 아빠가 회사에서 일하는 것은 기업이 이윤을 극대화하기 위해서 재화나 서비스를 만들어내는 생산 활동의 한 부분이야. 이 과정에서 네 아빠는 그 기여한 노동에 대한 대가로 임금을 받게 되는 거지.'},
 '담화3': {'질문': '그렇구나. 그럼 학교에서 쓰레기를 줍고, 교실을 청소하는 건 경제 활동에 속하지 않는 건가요? 우리가 용돈받는 것도 아니고...?',
  '답변': '음, 그것 자체는 직접적인 경제 활동이라고 볼 수 없지만, 짱구야, 학교 환경을 깨끗하게 유지하는 건 한 커뮤니티 내에서의 더 큰 사회적 가치를 생성하는 일이야. 경제 활동도 사회적 후생을 극대화하는 목적으로 정부가 하는 활동들과 밀접하게 연결되어 있으니까, 네가 하는 행동 역시 중요한 의미가 있어.'},
 '담화4': {'질문': '아! 선생님! 저희 집 강아지를 산책시키면서 친구들이랑 레몬에이드를 판매했어요. 이런 것도 ‘기업’에서 하는 ‘생산’ 활동 중 하나로 볼 수 있나요?',
  '답변': '많이 작지만 네가 레몬에이드를 만들어 판매한 것도 하나의 작은 기업 활동이라고 볼 수 있어, 짱구야. 상품을 만들어서 이윤을 얻으려는 너의 노력이 바로 기업의 생산 활동과 유사해. 창의적인 시도였어!'},
 '담화5': {'질문': '선생님! 저번에 친구랑 계란을 나눠먹었어요

In [21]:
qeustion = "앞에 설명한 개념을 다시 묻는 질문"
generator.make_q_a(script, student, teacher, qeustion, qeustion_num)

{'담화1': {'질문': '선생님, 생산이라는 건 물건을 만드는 것만 해당되는 건가요?',
  '답변': '아니, 짱구야. 생산에는 물건을 만드는 것뿐만 아니라 서비스를 제공하는 것도 포함돼. 예를 들어, 난 지금 교육 서비스를 제공하고 있기 때문에 생산 활동을 하고 있는 거란다.',
  '대답': '아하! 그럼 노래 부르는 가수나 병원에서 치료하는 의사도 생산자라는 거네요!'},
 '담화2': {'질문': '그럼 소비는 뭐에요? 물건 사는 거만 소비인가요?',
  '답변': '소비는 물건을 사는 것도 포함하지만, 자기 만족을 위해서 서비스를 이용하는 것도 소비야. 예를 들어, 햄버거를 사 먹는 것도 소비고, 놀이공원 티켓을 사서 즐기는 것도 소비란다.',
  '대답': '아! 그렇다면 영화 볼 때 표 사는 것도 소비하는 거구나!'},
 '담화3': {'질문': '분배가 또 뭐예요? 이게 왜 필요한 거죠?',
  '답변': '분배는 경제 활동에 참여해서 얻은 대가를 나누어 받는 것을 말해. 예를 들어, 일을 해서 받는 임금이나 토지를 빌려줘서 받는 임대료가 그렇겠지. 이렇게 분배를 통해 모두가 공평하게 대가를 받는 것이 중요해.',
  '대답': '오, 그래서 엄마가 회사 다니고 월급 받아오시는 거군요! 이해됐어요.'},
 '담화4': {'질문': '경제 순환 모형이라는 건 무슨 모델이에요? 어렵다...',
  '답변': '경제 순환 모형은 경제에서 발생하는 활동들이 어떻게 서로 연결되어 있는지를 보여주는 모델이야. 예를 들어, 가계에서는 노동력을 제공하고 기업은 그 노동력으로 상품이나 서비스를 만들어. 이렇게 서로 주고받는 관계가 경제를 순환하게 만드는 거지.',
  '대답': '아~ 그러니까 사람들과 회사들이 서로 도와가며 돌아간다는 거네요! 신기해요.'},
 '담화5': {'질문': '사람들이 왜 경제 활동을 해요? 그냥 놀면 안 되나요?',
  '답변': '사람들은 각자 필요하고 원하는 것을 충족시키기 위해서 경제 활동을 해. 먹을 것도 사야 하고, 

In [22]:
qeustion = "앞에 설명한 개념을 다시 묻는 질문"
generator.make_q_a(script_summary, student, teacher, qeustion, qeustion_num)

{'담화1': {'질문': '선생님, 경제 활동에는 뭐가 있지요?',
  '답변': '경제 활동에는 생산, 분배, 소비가 있단다, 짱구야. 생산은 재화나 서비스를 만드는 거고, 분배는 그 생산에 참여한 대가로 임금이나 이자를 받는 거야. 그리고 소비는 우리가 재화나 서비스를 사서 만족을 얻는 행위란다.',
  '대답': '아하, 그러면 생산부터 소비까지 다 연결되어 있겠네요!'},
 '담화2': {'질문': '가계랑 기업이랑 정부는 경제에서 어떤 일을 해요?',
  '답변': '가계는 소비를 주로 해서 만족을 얻으려고 하고, 기업은 재화나 서비스를 만들어 이윤을 얻으려고 해. 정부는 우리 사회의 후생을 더 좋게 만들기 위해 재정활동을 포함한 여러 경제 활동을 해.',
  '대답': '오, 그러면 세상이 돌아가게 하는 큰 역할들이네요!'},
 '담화3': {'질문': '해당 모형은 뭔가요?',
  '답변': '네가 말한 것은 경제 순환 모형이야. 이 모형은 가계, 기업, 정부가 어떻게 상호작용하는지 보여줘. 실물의 흐름과 화폐의 흐름을 포함해서, 생산물과 생산 요소 시장에서의 거래를 통해 이루어져.',
  '대답': '그러면 물건과 돈이 계속 돌고 도는 거군요!'},
 '담화4': {'질문': '선생님, 각각의 경제 주체들이 뭐를 목적으로 활동해요?',
  '답변': '가계는 효용을 극대화하려고 활동하고, 기업은 이윤을 극대화하기 위해 노력하며, 정부는 사회의 후생을 극대화하려고 여러 가지 경제 활동을 해.',
  '대답': '다들 제 각각 중요한 목표가 있네요.'},
 '담화5': {'질문': '왜 경제학을 배워야 해요?',
  '답변': '경제학을 배우면 우리 생활과 세상에서 일어나는 경제 현상들을 이해하고 해석하는 데 필요한 기초 지식을 얻게 돼. 이는 네가 앞으로 더 현명한 선택을 하는 데 도움이 될 거야.',
  '대답': '와, 그럼 경제학이 정말 중요하군요!'}}

In [23]:
qeustion = "컴퓨터를 구매하는 것은 어떤 경제적 활동인가요? 생산인가요?"
generator.answer_for_question(script, student, teacher, qeustion)

{'답변': "짱구야, 컴퓨터를 구매하는 것은 '소비'라는 경제 활동이란다. 왜냐하면, 컴퓨터를 사면서 기분 좋은 경험이나 만족감 같은 것을 얻기 위해서 사니까. 생산은 무언가를 만들거나 제공하는 활동을 말하는 거야. 예를 들어 컴퓨터를 만드는 회사에서 컴퓨터를 조립하고 판매하는 것이 생산 활동이지. 그러니까, 네가 컴퓨터를 구매하는 건, 만드는 사람이 아니라 사용할 목적으로 사는 거니까 소비 활동이 된단다."}

In [4]:
qeustion = "컴퓨터를 구매하는 것은 어떤 경제적 활동인가요? 생산인가요?"
answer = generator.answer_for_question(script_summary, student, teacher, qeustion)
answer

{'답변': "짱구야, 컴퓨터를 구매하는 건 '소비'라는 경제 활동이야. 컴퓨터는 이미 만들어진 제품이니까, 그걸 사서 사용하는 건 소비자인 우리가 만족을 얻기 위해 물건을 사용하는 거란다. 생산은 물건을 만드는 과정을 말하거든. 예를 들면 컴퓨터를 만드는 회사가 그걸 하는 거지. 그리고 우리가 컴퓨터를 삼으로써 그 회사에게 돈을 주고, 그 돈으로 그 회사는 더 많은 컴퓨터를 만들 수 있어. 이해가 되었니?"}

In [13]:
generator.answer_for_response(script_summary, teacher, student, "컴퓨터를 구매하는 것은 어떤 경제적 활동인가요? 생산인가요?", "선생님, 컴퓨터를 구매하는 것은 비용을 지불하고 원하는 재화를 얻는 것이기 때문에 선생님이 말씀하셨듯이 과일을 사먹는 것 처럼 생산에 해당하는 활동이에요.")

{'정답여부': '아니요',
 '답변': "짱구야, 컴퓨터를 구매하는 행위는 경제 활동 중에서 '소비'에 해당해. '생산'은 물건이나 서비스를 만들어 내는 활동을 말해. 컴퓨터를 구매하면 우리가 그 컴퓨터를 사용해서 여러 가지 일을 할 수 있어서 만족을 얻게 되지. 그래서 이런 구매 행위는 소비 활동의 한 예시라고 볼 수 있어. 이해됐어?"}

In [14]:
generator.answer_for_response(script_summary, student, teacher, "컴퓨터를 구매하는 것은 어떤 경제적 활동인가요? 생산인가요?", "컴퓨터를 구매하는 것은 비용을 지불하고 원하는 재화를 얻는 것이기 때문에 아까 설명했듯이 과일을 사먹는 것 처럼 생산에 해당하는 활동이에요.")

{'정답여부': '아니요',
 '답변': "채송화 선생님, 컴퓨터를 구매하는 건 '소비'하는 경제적 활동이에요. '생산'이라는 건 물건이나 서비스를 만드는 걸 말하죠. 예를 들어, 컴퓨터 공장에서 컴퓨터를 만드는 건 생산이에요. 그리고 우리가 마트나 인터넷에서 컴퓨터를 사서 쓰는 건 '소비'라고 해요. 공부할 때 컴퓨터 쓰는 것도 소비 중 하나죠. 선생님이 알려주신 거 잘 기억하고 있어요. 다음에도 재미있는 경제 이야기 많이 들려주세요!"}

### Done
* 스크립트를 주면 질문과 답변의 담화를 생성하는 기능
* 스크립트와 질문을 주면 답하는 기능
* 스크립트와 질문 및 답변을 주면 정답인지 체크하여 대답하는 기능

### 참고
* 스크립트를 주고 한 번에 질문과 답변 담화를 여러개 만들어 내어 효율적으로 동작 가능
* 학습 목표에 대해 질문할 지, 일상 얘기를 할 지, 농담을 할 지 전달해줘야 할 듯 함
* 학습자와 교수자의 페르소나를 잘 정의하여 넘겨줘야 할 듯
* gpt 한테 스크립트를 요약시켜 내용을 줄이고 컨텍스트로 전달해도 잘 되는 듯 함
* 학습자와 교수자의 역할이 복잡해 지는 경우 따로 클래스를 구분하여 구조화 하는 것이 좋을 수도
* 전체 담화의 시나리오 유형을 정하는 것도 좋을 듯 하다. ex) 질문자가 잘못된 추론을 한다. 교수자가 올바르게 정정해준다. 질문자가 올바르게 재 추론 한다.


### What to do
* 학습자 연령과 수준에 따른 질문 수준이 얼마나 다양하게 만들어지는지 확인
* 프롬프트 개선 (구체적이고 효과적인 조건 개발 및 추가)
* 질문 생성 시 전체 스크립트에 대해 고루 다룰 수 있도록 보완